In [2]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../fastai/") # go to parent dir

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *



    Only loading the 'en' tokenizer.



In [3]:
PATH='./jos/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

test/  train/


In [29]:
## organizing data
#import os
#src = './jos'
# for fname in os.listdir(src):
#       if os.path.isfile(os.path.join(src, fname)):
#             if random.random()>0.8:
#                 os.rename(os.path.join(src, fname), os.path.join(src+'/train', fname))
#             else:
#                 os.rename(os.path.join(src, fname), os.path.join(src+'/test', fname))

In [37]:
trn_files = !ls {TRN}
trn_files[:10]

['Jos0301a-Missa_Ave_maris_stella-Kyrie.krn',
 'Jos0301b-Missa_Ave_maris_stella-Gloria.krn',
 'Jos0301c-Missa_Ave_maris_stella-Credo.krn',
 'Jos0301d-Missa_Ave_maris_stella-Sanctus.krn',
 'Jos0301e-Missa_Ave_maris_stella-Agnus.krn',
 'Jos0302c-Missa_Da_pacem-Credo.krn',
 'Jos0302e-Missa_Da_pacem-Agnus.krn',
 'Jos0303b-Missa_De_beata_virgine-Gloria.krn',
 'Jos0303c-Missa_De_beata_virgine-Credo.krn',
 'Jos0303d-Missa_De_beata_virgine-Sanctus.krn']

In [43]:
!find {TRN} -name '*.krn' | xargs cat | wc -w

1172744


In [4]:
!find {VAL} -name '*.krn' | xargs cat | wc -w

295493


In [5]:
TEXT = data.Field()

In [7]:
bs=64; bptt=70

In [8]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [10]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(260, 2371, 1, 1173100)

In [13]:
TEXT.vocab.itos[:12]

['<unk>',
 '<pad>',
 '.',
 '0r',
 '1r',
 '2r',
 '2c\\',
 '1A',
 '1d',
 '1G',
 '2d\\',
 '1c']

In [14]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [15]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [16]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [17]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)


HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.013943   3.683829  
    1      2.858282   2.567854                              
    2      2.436526   2.277471                              
    3      2.10973    2.006981                              
    4      1.797876   1.800848                              
    5      1.660158   1.695154                              
    6      1.614569   1.656632                              
    7      1.611573   1.632987                              
    8      1.454498   1.528905                              
    9      1.347518   1.484467                              
    10     1.254057   1.406046                              
    11     1.184418   1.373622                              
    12     1.13593    1.363384                              
    13     1.107699   1.348986                              
    14     1.089399   1.34737                               



[1.3473704]

In [26]:

m=learner.model
s="""!!!COM:	Japart, Jean
!!!CDT:	~1450//-~1507//
!!!OTL:	Famene un poco"""
t=TEXT.numericalize(s)
' '.join(s[0])

'!'

In [27]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [28]:

nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['<unk>', '!', '.', '*', 'and', 'at', '==', 'a', 'the', 'in']

In [29]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

!!!COM:	Japart, Jean
!!!CDT:	~1450//-~1507//
!!!OTL:	Famene un poco
 

! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ...


In [25]:
print(ss,"\n")
for i in range(500):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

!!!COM:	Japart, Jean
!!!CDT:	~1450//-~1507//
!!!OTL:	Famene un poco
 

! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !